In [ ]:
# gtx5070ti를 지원하는 cuda12.8버전 지정 설치
#pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

In [7]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")

GPU is available!
Device name: NVIDIA GeForce RTX 5070 Ti


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 1. GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. 하이퍼파라미터 설정 (모델 학습에 필요한 설정값들)
batch_size = 64
learning_rate = 0.001
epochs = 10

# 3. 데이터 로더 준비
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 4. 신경망 모델 정의
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # 첫 번째 레이어: 입력층 (28*28) -> 은닉층 128개
        self.fc1 = nn.Linear(28 * 28, 128)
        # 두 번째 레이어: 은닉층 128개 -> 은닉층 64개
        self.fc2 = nn.Linear(128, 64)
        # 세 번째 레이어: 은닉층 64개 -> 출력층 10개 (0~9까지의 숫자)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        # 텐서를 1차원으로 펼치기 (Flatten)
        x = x.view(-1, 28 * 28)
        # 활성화 함수 ReLU를 적용
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        # 마지막 출력층
        x = self.fc3(x)
        return x 

    def forward(self, x):
        # 데이터가 모델을 통과하는 과정을 정의해야 함
        return x

# 5. 모델, 손실 함수, 옵티마이저 설정
model = SimpleNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 6. 학습 루프 구현
def train(model, device, train_loader, optimizer, criterion):
    model.train() # 모델을 학습 모드로 설정
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) # 데이터를 GPU로 이동
        optimizer.zero_grad() # 기울기 초기화
        output = model(data) # 순전파
        loss = criterion(output, target) # 손실 계산
        loss.backward() # 역전파
        optimizer.step() # 가중치 업데이트

        if batch_idx % 100 == 0:
            print(f'Train Epoch: [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 7. 평가 함수 구현
def evaluate(model, device, test_loader):
    model.eval() # 모델을 평가 모드로 설정
    test_loss = 0
    correct = 0
    with torch.no_grad(): # 기울기 계산 비활성화 (메모리 절약)
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # 배치별 손실 누적
            pred = output.argmax(dim=1, keepdim=True) # 가장 높은 확률의 클래스 선택
            correct += pred.eq(target.view_as(pred)).sum().item() # 정답 개수 계산

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

# 8. 메인 함수
if __name__ == "__main__":
    for epoch in range(1, epochs + 1):
        print(f"--- Epoch {epoch}/{epochs} ---")
        train(model, device, train_loader, optimizer, criterion)
        evaluate(model, device, test_loader)

Using device: cuda
--- Epoch 1/10 ---


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
